# ELS Pricer - GPU Benchmark on Google Colab (Extended)

**C++/CUDA 기반 2자산 Step-Down ELS Pricer 성능 테스트 - 확장판**

이 노트북은 다음을 테스트합니다:
- ✅ CPU 버전 (6가지 그리드 크기)
- ✅ GPU 버전 (6가지 그리드 크기)
- ✅ CPU vs GPU 성능 비교
- ✅ 버그 수정된 정확한 가격 계산
- 🆕 **Time Step (Nt) 스케일링 테스트**
- 🆕 **Spatial Grid 스케일링 테스트**

## 🔧 최근 업데이트 (2024-11-14)
- **조기상환 버그 수정**: 강제 상환으로 변경 (이전: 선택적)
- **타임스텝 버그 수정**: 만기 관측일 포함
- **가격 정확도 향상**: 111.7원 → 103.9원 (오차 7% → 0.5%)

## 📋 테스트 그리드 크기

| # | N1 | N2 | Nt | 총 Points | 설명 |
|---|----|----|----|-----------| |
| 1 | 100 | 100 | 200 | 2M | Small - Medium density |
| 2 | 100 | 100 | 1000 | 10M | Small - High time resolution |
| 3 | 200 | 200 | 200 | 8M | Large - Medium density |
| 4 | 200 | 200 | 1000 | 40M | Large - High time resolution |
| 5 | 400 | 400 | 200 | 32M | Very Large - Medium density |
| 6 | 400 | 400 | 1000 | 160M | Very Large - High time resolution |

---

## Step 1: GPU 환경 확인

In [ ]:
# GPU 확인
!nvidia-smi
print("\n" + "="*70)
!nvcc --version

## Step 2: 필요한 도구 설치

In [ ]:
# CMake와 빌드 도구 설치
!apt-get update -qq
!apt-get install -y -qq cmake build-essential

# 버전 확인
!cmake --version
!g++ --version | head -1

## Step 3: 프로젝트 업로드

**방법 1: 파일 직접 업로드 (추천)**
- `els-pricer-colab.tar.gz` 파일을 준비 (버그 수정 버전)
- 아래 셀 실행 후 파일 선택

**방법 2: Google Drive 마운트**
- Drive에 미리 업로드한 경우

**📌 중요**: 2024-11-14 이후 생성된 `els-pricer-colab.tar.gz` 파일을 사용하세요 (버그 수정 포함)

In [ ]:
# 방법 1: 파일 업로드
from google.colab import files
import os

print("📁 els-pricer-colab.tar.gz 파일을 업로드해주세요...")
print("   (버그 수정 버전: 2024-11-14 이후 생성)")
uploaded = files.upload()

# 압축 해제
if 'els-pricer-colab.tar.gz' in uploaded:
    !tar -xzf els-pricer-colab.tar.gz
    print("\n✓ 프로젝트 압축 해제 완료!")
    !ls -la els-pricer-cpp/
elif 'els-pricer-cpp.tar.gz' in uploaded:
    # 구버전 파일명도 지원
    !tar -xzf els-pricer-cpp.tar.gz
    print("\n✓ 프로젝트 압축 해제 완료! (구버전 파일명)")
    !ls -la els-pricer-cpp/
else:
    print("❌ els-pricer-colab.tar.gz 파일을 찾을 수 없습니다.")

In [ ]:
# (선택사항) 방법 2: Google Drive에서 가져오기
# from google.colab import drive
# drive.mount('/content/drive')
# !cp /content/drive/MyDrive/els-pricer-colab.tar.gz .
# !tar -xzf els-pricer-colab.tar.gz

## Step 4: 프로젝트 빌드 (GPU 버전)

In [ ]:
import os

# 프로젝트 디렉토리 확인
project_dir = '/content/els-pricer-cpp'
if not os.path.exists(project_dir):
    print("❌ 프로젝트 디렉토리를 찾을 수 없습니다!")
    print("   Step 3에서 파일을 업로드하고 압축을 해제했는지 확인하세요.")
else:
    print("✓ 프로젝트 디렉토리 확인됨")
    
    # 작업 디렉토리 변경
    os.chdir(project_dir)
    print(f"✓ 작업 디렉토리: {os.getcwd()}")
    
    # 빌드 디렉토리 생성
    !rm -rf build
    !mkdir -p build
    
    # 빌드 디렉토리로 이동
    os.chdir(os.path.join(project_dir, 'build'))
    print(f"✓ 빌드 디렉토리: {os.getcwd()}")
    
    # CMake 설정 (GPU 포함)
    print("\n🔧 CMake 설정 중...\n")
    !cmake ..
    
    print("\n" + "="*70)
    print("🔨 컴파일 중 (병렬 빌드)...\n")
    !make -j4
    
    print("\n" + "="*70)
    print("✓ 빌드 완료!")
    !ls -lh els_pricer test_els benchmark_cpu_vs_gpu validate_price 2>/dev/null || echo "일부 실행 파일이 없을 수 있습니다."

## Step 5: CPU vs GPU 기본 벤치마크 실행

빌드가 완료되었습니다! 이제 CPU와 GPU 성능을 직접 비교하는 벤치마크를 실행합니다.

**테스트할 그리드:**
- 100×100×200, 100×100×1000
- 200×200×200, 200×200×1000
- 400×400×200, 400×400×1000

각 그리드마다 CPU와 GPU 모두에서 실행하고 어느 쪽이 더 빠른지 확인합니다.

In [ ]:
import os

# 빌드 디렉토리로 이동
build_dir = '/content/els-pricer-cpp/build'
if os.path.exists(build_dir):
    os.chdir(build_dir)
    print(f"✓ 작업 디렉토리: {os.getcwd()}")
    
    print("\n🚀 CPU vs GPU 종합 벤치마크 시작!\n")
    print("="*70 + "\n")
    
    # CPU vs GPU 비교 프로그램 실행 (6개 그리드 모두 테스트)
    if os.path.exists('./benchmark_cpu_vs_gpu'):
        !./benchmark_cpu_vs_gpu
    else:
        print("⚠️ benchmark_cpu_vs_gpu 실행 파일을 찾을 수 없습니다.")
        print("   대신 els_pricer로 간단한 테스트를 실행합니다.")
        if os.path.exists('./els_pricer'):
            !./els_pricer
        else:
            print("❌ 실행 파일이 없습니다. 빌드가 성공적으로 완료되었는지 확인하세요.")
else:
    print("❌ 빌드 디렉토리를 찾을 수 없습니다. Step 4를 먼저 실행하세요.")

## Step 6: 결과 데이터 확인 및 분석

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# CSV 파일 경로
csv_path = '/content/els-pricer-cpp/build/cpu_vs_gpu_results.csv'

if os.path.exists(csv_path):
    # CSV 파일 읽기
    df = pd.read_csv(csv_path)
    
    print("📊 CPU vs GPU 벤치마크 결과:\n")
    print(df.to_string(index=False))
    
    print("\n" + "═"*70)
    print("📈 통계 요약:")
    print(f"   평균 CPU 시간: {df['CPU_Time'].mean():.3f}초")
    print(f"   평균 GPU 시간: {df['GPU_Time'].mean():.3f}초")
    print(f"   평균 가속 비율: {df['Speedup'].mean():.2f}×")
    print(f"   최대 가속 비율: {df['Speedup'].max():.2f}×")
    print(f"   GPU 승리 횟수: {(df['Speedup'] > 1.0).sum()}/6")
    print(f"   CPU 승리 횟수: {(df['Speedup'] < 1.0).sum()}/6")
    
    # 각 그리드별 승자 표시
    print("\n🏆 그리드별 승자:")
    for idx, row in df.iterrows():
        grid_name = f"{row['Grid_N1']}×{row['Grid_N2']}×{row['Grid_Nt']}"
        if row['Speedup'] > 1.0:
            winner = f"GPU ✓ ({row['Speedup']:.2f}× 빠름)"
        elif row['Speedup'] < 1.0:
            winner = f"CPU ✓ ({1.0/row['Speedup']:.2f}× 빠름)"
        else:
            winner = "무승부"
        print(f"   {grid_name:16s} → {winner}")
else:
    print("⚠️ 벤치마크 결과 CSV 파일을 찾을 수 없습니다.")
    print("   Step 5에서 벤치마크를 먼저 실행하세요.")

## Step 7: 성능 비교 시각화

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

csv_path = '/content/els-pricer-cpp/build/cpu_vs_gpu_results.csv'

if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    
    # 그리드 라벨 생성
    grid_labels = [f"{row['Grid_N1']}×{row['Grid_N2']}×{row['Grid_Nt']}" 
                   for _, row in df.iterrows()]
    
    # 플롯 생성
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('ELS Pricer: CPU vs GPU Performance Comparison', fontsize=16, fontweight='bold')
    
    # 1. CPU vs GPU 실행 시간 비교
    ax1 = axes[0, 0]
    x = np.arange(len(grid_labels))
    width = 0.35
    bars1 = ax1.bar(x - width/2, df['CPU_Time'], width, label='CPU', color='steelblue', alpha=0.8)
    bars2 = ax1.bar(x + width/2, df['GPU_Time'], width, label='GPU', color='coral', alpha=0.8)
    ax1.set_xlabel('Grid Size', fontweight='bold')
    ax1.set_ylabel('Time (seconds)', fontweight='bold')
    ax1.set_title('Execution Time Comparison')
    ax1.set_xticks(x)
    ax1.set_xticklabels(grid_labels, rotation=45, ha='right')
    ax1.legend()
    ax1.grid(axis='y', alpha=0.3)
    
    # 2. 가속 비율
    ax2 = axes[0, 1]
    colors = ['green' if s > 1.0 else 'red' for s in df['Speedup']]
    bars = ax2.bar(grid_labels, df['Speedup'], color=colors, alpha=0.7)
    ax2.axhline(y=1.0, color='black', linestyle='--', linewidth=1, label='Break-even')
    ax2.set_xlabel('Grid Size', fontweight='bold')
    ax2.set_ylabel('Speedup (×)', fontweight='bold')
    ax2.set_title('GPU Speedup over CPU')
    ax2.set_xticklabels(grid_labels, rotation=45, ha='right')
    ax2.legend()
    ax2.grid(axis='y', alpha=0.3)
    
    # 가속비 값 표시
    for i, (label, speedup) in enumerate(zip(grid_labels, df['Speedup'])):
        ax2.text(i, speedup + 0.5, f'{speedup:.1f}×', 
                 ha='center', va='bottom', fontsize=9, fontweight='bold')
    
    # 3. 가격 수렴성
    ax3 = axes[1, 0]
    ax3.plot(grid_labels, df['CPU_Price'], 'o-', label='CPU Price', markersize=8, linewidth=2)
    ax3.plot(grid_labels, df['GPU_Price'], 's-', label='GPU Price', markersize=8, linewidth=2)
    ax3.set_xlabel('Grid Size', fontweight='bold')
    ax3.set_ylabel('ELS Price', fontweight='bold')
    ax3.set_title('Price Convergence')
    ax3.set_xticklabels(grid_labels, rotation=45, ha='right')
    ax3.legend()
    ax3.grid(alpha=0.3)
    
    # 4. Points/second 처리 속도
    ax4 = axes[1, 1]
    cpu_throughput = df['Total_Points'] / df['CPU_Time'] / 1e6
    gpu_throughput = df['Total_Points'] / df['GPU_Time'] / 1e6
    x = np.arange(len(grid_labels))
    bars1 = ax4.bar(x - width/2, cpu_throughput, width, label='CPU', color='steelblue', alpha=0.8)
    bars2 = ax4.bar(x + width/2, gpu_throughput, width, label='GPU', color='coral', alpha=0.8)
    ax4.set_xlabel('Grid Size', fontweight='bold')
    ax4.set_ylabel('Throughput (M points/sec)', fontweight='bold')
    ax4.set_title('Processing Throughput')
    ax4.set_xticks(x)
    ax4.set_xticklabels(grid_labels, rotation=45, ha='right')
    ax4.legend()
    ax4.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('cpu_vs_gpu_benchmark.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\n✅ 그래프가 cpu_vs_gpu_benchmark.png로 저장되었습니다!")
else:
    print("⚠️ 시각화할 데이터가 없습니다. Step 5를 먼저 실행하세요.")

## Step 8: Time Step (Nt) 스케일링 테스트 🆕

**100×100 그리드 고정**, Time Step (Nt)을 200부터 2000까지 증가시키면서 CPU와 GPU 성능 비교

테스트 구성: Nt = 200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000

In [ ]:
import os
import time
import subprocess
import pandas as pd
import numpy as np

build_dir = '/content/els-pricer-cpp/build'
if os.path.exists(build_dir):
    os.chdir(build_dir)
    
    print("🔬 Time Step (Nt) 스케일링 테스트\n")
    print("="*70)
    print("Grid: 100×100 (고정)")
    print("Nt: 200 → 2000 (10 steps)\n")
    
    # 테스트할 Nt 값들
    nt_values = [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
    
    results = []
    
    for nt in nt_values:
        print(f"\n[테스트] 100×100×{nt}...")
        
        # 커스텀 테스트 프로그램 생성 (C++ 코드)
        test_code = f"""
#include "Grid2D.h"
#include "ELSProduct.h"
#include "ADISolver.h"
#ifdef __CUDACC__
#include "CUDAADISolver.cuh"
#endif
#include <iostream>
#include <chrono>

using namespace ELSPricer;

int main() {{
    auto product = createSampleELS();
    
    int N1 = 100, N2 = 100, Nt = {nt};
    
    // CPU 테스트
    auto cpu_result = priceELS(product, N1, N2, Nt, false);
    
    // GPU 테스트 (if available)
    double gpu_time = 0.0;
    double gpu_price = 0.0;
#ifdef __CUDACC__
    auto gpu_result = CUDA::priceELSGPU(product, N1, N2, Nt, false);
    gpu_time = gpu_result.computeTime;
    gpu_price = gpu_result.price;
#endif
    
    std::cout << N1 << "," << N2 << "," << Nt << ","
              << cpu_result.computeTime << "," << cpu_result.price << ","
              << gpu_time << "," << gpu_price << std::endl;
    
    return 0;
}}
        """
        
        # 파일 작성
        with open('test_nt_scaling.cpp', 'w') as f:
            f.write(test_code)
        
        # 컴파일
        compile_cmd = [
            'g++', '-std=c++17', '-O3',
            '-I../include',
            'test_nt_scaling.cpp',
            'libels_pricer_lib.a',
            '-o', 'test_nt_scaling'
        ]
        
        try:
            subprocess.run(compile_cmd, check=True, capture_output=True)
            
            # 실행
            result = subprocess.run('./test_nt_scaling', capture_output=True, text=True)
            if result.returncode == 0:
                data = result.stdout.strip().split(',')
                results.append({
                    'N1': int(data[0]),
                    'N2': int(data[1]),
                    'Nt': int(data[2]),
                    'CPU_Time': float(data[3]),
                    'CPU_Price': float(data[4]),
                    'GPU_Time': float(data[5]) if len(data) > 5 and float(data[5]) > 0 else None,
                    'GPU_Price': float(data[6]) if len(data) > 6 else None
                })
                
                cpu_time = float(data[3])
                gpu_time = float(data[5]) if len(data) > 5 and float(data[5]) > 0 else None
                
                if gpu_time:
                    speedup = cpu_time / gpu_time
                    print(f"   ✓ CPU: {cpu_time:.3f}s, GPU: {gpu_time:.3f}s, Speedup: {speedup:.2f}×")
                else:
                    print(f"   ✓ CPU: {cpu_time:.3f}s, GPU: N/A")
        except Exception as e:
            print(f"   ❌ 테스트 실패: {e}")
    
    # DataFrame 생성
    if results:
        df_nt = pd.DataFrame(results)
        df_nt['Speedup'] = df_nt.apply(
            lambda row: row['CPU_Time'] / row['GPU_Time'] if row['GPU_Time'] else None, 
            axis=1
        )
        
        # CSV 저장
        df_nt.to_csv('nt_scaling_results.csv', index=False)
        
        print("\n" + "="*70)
        print("\n📊 Nt 스케일링 결과 요약:\n")
        print(df_nt[['Nt', 'CPU_Time', 'GPU_Time', 'Speedup']].to_string(index=False))
        
        # 통계
        print("\n📈 통계:")
        print(f"   CPU 시간 증가율: {df_nt['CPU_Time'].iloc[-1] / df_nt['CPU_Time'].iloc[0]:.2f}× (Nt {nt_values[0]} → {nt_values[-1]})")
        if df_nt['GPU_Time'].notna().any():
            print(f"   GPU 시간 증가율: {df_nt['GPU_Time'].iloc[-1] / df_nt['GPU_Time'].iloc[0]:.2f}× (Nt {nt_values[0]} → {nt_values[-1]})")
            print(f"   평균 Speedup: {df_nt['Speedup'].mean():.2f}×")
            print(f"   최대 Speedup: {df_nt['Speedup'].max():.2f}× (Nt={df_nt.loc[df_nt['Speedup'].idxmax(), 'Nt']})")
            print(f"   최소 Speedup: {df_nt['Speedup'].min():.2f}× (Nt={df_nt.loc[df_nt['Speedup'].idxmin(), 'Nt']})")
    else:
        print("❌ 테스트 결과가 없습니다.")
else:
    print("❌ 빌드 디렉토리를 찾을 수 없습니다.")

## Step 9: Nt 스케일링 시각화 🆕

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

csv_path = '/content/els-pricer-cpp/build/nt_scaling_results.csv'

if os.path.exists(csv_path):
    df_nt = pd.read_csv(csv_path)
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Time Step (Nt) Scaling Analysis - 100×100 Grid Fixed', fontsize=16, fontweight='bold')
    
    # 1. CPU vs GPU 시간 비교
    ax1 = axes[0, 0]
    ax1.plot(df_nt['Nt'], df_nt['CPU_Time'], 'o-', label='CPU', linewidth=2, markersize=8, color='steelblue')
    if df_nt['GPU_Time'].notna().any():
        ax1.plot(df_nt['Nt'], df_nt['GPU_Time'], 's-', label='GPU', linewidth=2, markersize=8, color='coral')
    ax1.set_xlabel('Time Steps (Nt)', fontweight='bold')
    ax1.set_ylabel('Execution Time (seconds)', fontweight='bold')
    ax1.set_title('Execution Time vs Nt')
    ax1.legend()
    ax1.grid(alpha=0.3)
    
    # 2. Speedup vs Nt
    ax2 = axes[0, 1]
    if 'Speedup' in df_nt.columns and df_nt['Speedup'].notna().any():
        colors = ['green' if s > 1.0 else 'red' for s in df_nt['Speedup']]
        ax2.bar(df_nt['Nt'].astype(str), df_nt['Speedup'], color=colors, alpha=0.7)
        ax2.axhline(y=1.0, color='black', linestyle='--', linewidth=1, label='Break-even')
        ax2.set_xlabel('Time Steps (Nt)', fontweight='bold')
        ax2.set_ylabel('Speedup (CPU/GPU)', fontweight='bold')
        ax2.set_title('GPU Speedup vs Nt')
        ax2.legend()
        ax2.grid(axis='y', alpha=0.3)
        
        # Speedup 값 표시
        for i, (nt, speedup) in enumerate(zip(df_nt['Nt'], df_nt['Speedup'])):
            if pd.notna(speedup):
                ax2.text(i, speedup + 0.05, f'{speedup:.2f}×', 
                         ha='center', va='bottom', fontsize=8)
    
    # 3. 타임스텝당 시간 (효율성)
    ax3 = axes[1, 0]
    cpu_per_step = (df_nt['CPU_Time'] / df_nt['Nt']) * 1000  # ms
    ax3.plot(df_nt['Nt'], cpu_per_step, 'o-', label='CPU', linewidth=2, markersize=8, color='steelblue')
    if df_nt['GPU_Time'].notna().any():
        gpu_per_step = (df_nt['GPU_Time'] / df_nt['Nt']) * 1000  # ms
        ax3.plot(df_nt['Nt'], gpu_per_step, 's-', label='GPU', linewidth=2, markersize=8, color='coral')
    ax3.set_xlabel('Time Steps (Nt)', fontweight='bold')
    ax3.set_ylabel('Time per Step (ms)', fontweight='bold')
    ax3.set_title('Efficiency: Time per Timestep')
    ax3.legend()
    ax3.grid(alpha=0.3)
    
    # 4. Log-scale 시간 비교
    ax4 = axes[1, 1]
    ax4.loglog(df_nt['Nt'], df_nt['CPU_Time'], 'o-', label='CPU', linewidth=2, markersize=8, color='steelblue')
    if df_nt['GPU_Time'].notna().any():
        ax4.loglog(df_nt['Nt'], df_nt['GPU_Time'], 's-', label='GPU', linewidth=2, markersize=8, color='coral')
    
    # 선형 스케일링 참조선
    nt_ref = df_nt['Nt'].values
    linear_ref = df_nt['CPU_Time'].iloc[0] * (nt_ref / nt_ref[0])
    ax4.loglog(nt_ref, linear_ref, '--', color='gray', alpha=0.5, label='Linear scaling')
    
    ax4.set_xlabel('Time Steps (Nt) [log scale]', fontweight='bold')
    ax4.set_ylabel('Execution Time (seconds) [log scale]', fontweight='bold')
    ax4.set_title('Scaling Behavior (Log-Log Plot)')
    ax4.legend()
    ax4.grid(alpha=0.3, which='both')
    
    plt.tight_layout()
    plt.savefig('nt_scaling_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\n✅ 그래프가 nt_scaling_analysis.png로 저장되었습니다!")
else:
    print("⚠️ Nt 스케일링 데이터가 없습니다. Step 8을 먼저 실행하세요.")

## Step 10: Spatial Grid 스케일링 테스트 🆕

**Nt=1000 고정**, Spatial Grid를 100×100부터 700×700까지 증가시키면서 CPU와 GPU 성능 비교

테스트 구성: 100×100, 200×200, 300×300, 400×400, 500×500, 600×600, 700×700

In [ ]:
import os
import time
import subprocess
import pandas as pd
import numpy as np

build_dir = '/content/els-pricer-cpp/build'
if os.path.exists(build_dir):
    os.chdir(build_dir)
    
    print("🔬 Spatial Grid 스케일링 테스트\n")
    print("="*70)
    print("Nt: 1000 (고정)")
    print("Grid: 100×100 → 700×700 (7 steps)\n")
    
    # 테스트할 그리드 크기
    grid_sizes = [100, 200, 300, 400, 500, 600, 700]
    
    results = []
    
    for N in grid_sizes:
        print(f"\n[테스트] {N}×{N}×1000...")
        
        # 커스텀 테스트 프로그램 생성
        test_code = f"""
#include "Grid2D.h"
#include "ELSProduct.h"
#include "ADISolver.h"
#ifdef __CUDACC__
#include "CUDAADISolver.cuh"
#endif
#include <iostream>
#include <chrono>

using namespace ELSPricer;

int main() {{
    auto product = createSampleELS();
    
    int N1 = {N}, N2 = {N}, Nt = 1000;
    
    // CPU 테스트
    auto cpu_result = priceELS(product, N1, N2, Nt, false);
    
    // GPU 테스트 (if available)
    double gpu_time = 0.0;
    double gpu_price = 0.0;
#ifdef __CUDACC__
    auto gpu_result = CUDA::priceELSGPU(product, N1, N2, Nt, false);
    gpu_time = gpu_result.computeTime;
    gpu_price = gpu_result.price;
#endif
    
    std::cout << N1 << "," << N2 << "," << Nt << ","
              << cpu_result.computeTime << "," << cpu_result.price << ","
              << gpu_time << "," << gpu_price << std::endl;
    
    return 0;
}}
        """
        
        # 파일 작성
        with open('test_grid_scaling.cpp', 'w') as f:
            f.write(test_code)
        
        # 컴파일
        compile_cmd = [
            'g++', '-std=c++17', '-O3',
            '-I../include',
            'test_grid_scaling.cpp',
            'libels_pricer_lib.a',
            '-o', 'test_grid_scaling'
        ]
        
        try:
            subprocess.run(compile_cmd, check=True, capture_output=True)
            
            # 실행 (큰 그리드는 시간이 오래 걸릴 수 있음)
            result = subprocess.run('./test_grid_scaling', capture_output=True, text=True, timeout=300)
            if result.returncode == 0:
                data = result.stdout.strip().split(',')
                results.append({
                    'N1': int(data[0]),
                    'N2': int(data[1]),
                    'Nt': int(data[2]),
                    'Grid_Size': f"{data[0]}×{data[1]}",
                    'Total_Points': int(data[0]) * int(data[1]) * int(data[2]),
                    'CPU_Time': float(data[3]),
                    'CPU_Price': float(data[4]),
                    'GPU_Time': float(data[5]) if len(data) > 5 and float(data[5]) > 0 else None,
                    'GPU_Price': float(data[6]) if len(data) > 6 else None
                })
                
                cpu_time = float(data[3])
                gpu_time = float(data[5]) if len(data) > 5 and float(data[5]) > 0 else None
                total_points = int(data[0]) * int(data[1]) * int(data[2])
                
                if gpu_time:
                    speedup = cpu_time / gpu_time
                    print(f"   ✓ CPU: {cpu_time:.3f}s, GPU: {gpu_time:.3f}s, Speedup: {speedup:.2f}×")
                    print(f"      Total Points: {total_points/1e6:.1f}M")
                else:
                    print(f"   ✓ CPU: {cpu_time:.3f}s, GPU: N/A")
                    print(f"      Total Points: {total_points/1e6:.1f}M")
        except subprocess.TimeoutExpired:
            print(f"   ⚠️ 타임아웃 (300초 초과)")
        except Exception as e:
            print(f"   ❌ 테스트 실패: {e}")
    
    # DataFrame 생성
    if results:
        df_grid = pd.DataFrame(results)
        df_grid['Speedup'] = df_grid.apply(
            lambda row: row['CPU_Time'] / row['GPU_Time'] if row['GPU_Time'] else None, 
            axis=1
        )
        
        # CSV 저장
        df_grid.to_csv('grid_scaling_results.csv', index=False)
        
        print("\n" + "="*70)
        print("\n📊 Grid 스케일링 결과 요약:\n")
        print(df_grid[['Grid_Size', 'Total_Points', 'CPU_Time', 'GPU_Time', 'Speedup']].to_string(index=False))
        
        # 통계
        print("\n📈 통계:")
        print(f"   CPU 시간 증가율: {df_grid['CPU_Time'].iloc[-1] / df_grid['CPU_Time'].iloc[0]:.2f}× ({grid_sizes[0]}×{grid_sizes[0]} → {grid_sizes[-1]}×{grid_sizes[-1]})")
        if df_grid['GPU_Time'].notna().any():
            print(f"   GPU 시간 증가율: {df_grid['GPU_Time'].iloc[-1] / df_grid['GPU_Time'].iloc[0]:.2f}× ({grid_sizes[0]}×{grid_sizes[0]} → {grid_sizes[-1]}×{grid_sizes[-1]})")
            print(f"   평균 Speedup: {df_grid['Speedup'].mean():.2f}×")
            print(f"   최대 Speedup: {df_grid['Speedup'].max():.2f}× (Grid: {df_grid.loc[df_grid['Speedup'].idxmax(), 'Grid_Size']})")
            print(f"   최소 Speedup: {df_grid['Speedup'].min():.2f}× (Grid: {df_grid.loc[df_grid['Speedup'].idxmin(), 'Grid_Size']})")
    else:
        print("❌ 테스트 결과가 없습니다.")
else:
    print("❌ 빌드 디렉토리를 찾을 수 없습니다.")

## Step 11: Spatial Grid 스케일링 시각화 🆕

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

csv_path = '/content/els-pricer-cpp/build/grid_scaling_results.csv'

if os.path.exists(csv_path):
    df_grid = pd.read_csv(csv_path)
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Spatial Grid Scaling Analysis - Nt=1000 Fixed', fontsize=16, fontweight='bold')
    
    # 1. CPU vs GPU 시간 비교
    ax1 = axes[0, 0]
    ax1.plot(df_grid['N1'], df_grid['CPU_Time'], 'o-', label='CPU', linewidth=2, markersize=8, color='steelblue')
    if df_grid['GPU_Time'].notna().any():
        ax1.plot(df_grid['N1'], df_grid['GPU_Time'], 's-', label='GPU', linewidth=2, markersize=8, color='coral')
    ax1.set_xlabel('Grid Size (N×N)', fontweight='bold')
    ax1.set_ylabel('Execution Time (seconds)', fontweight='bold')
    ax1.set_title('Execution Time vs Grid Size')
    ax1.legend()
    ax1.grid(alpha=0.3)
    
    # 2. Speedup vs Grid Size
    ax2 = axes[0, 1]
    if 'Speedup' in df_grid.columns and df_grid['Speedup'].notna().any():
        ax2.plot(df_grid['N1'], df_grid['Speedup'], 'o-', linewidth=2, markersize=8, color='green')
        ax2.axhline(y=1.0, color='black', linestyle='--', linewidth=1, label='Break-even')
        ax2.set_xlabel('Grid Size (N×N)', fontweight='bold')
        ax2.set_ylabel('Speedup (CPU/GPU)', fontweight='bold')
        ax2.set_title('GPU Speedup vs Grid Size')
        ax2.legend()
        ax2.grid(alpha=0.3)
        
        # Speedup 값 표시
        for n, speedup in zip(df_grid['N1'], df_grid['Speedup']):
            if pd.notna(speedup):
                ax2.annotate(f'{speedup:.1f}×', 
                           xy=(n, speedup), 
                           xytext=(5, 5), 
                           textcoords='offset points',
                           fontsize=8)
    
    # 3. 처리량 (Points/sec)
    ax3 = axes[1, 0]
    cpu_throughput = df_grid['Total_Points'] / df_grid['CPU_Time'] / 1e6  # M points/sec
    ax3.plot(df_grid['N1'], cpu_throughput, 'o-', label='CPU', linewidth=2, markersize=8, color='steelblue')
    if df_grid['GPU_Time'].notna().any():
        gpu_throughput = df_grid['Total_Points'] / df_grid['GPU_Time'] / 1e6
        ax3.plot(df_grid['N1'], gpu_throughput, 's-', label='GPU', linewidth=2, markersize=8, color='coral')
    ax3.set_xlabel('Grid Size (N×N)', fontweight='bold')
    ax3.set_ylabel('Throughput (M points/sec)', fontweight='bold')
    ax3.set_title('Processing Throughput vs Grid Size')
    ax3.legend()
    ax3.grid(alpha=0.3)
    
    # 4. Log-Log 플롯 (스케일링 행동)
    ax4 = axes[1, 1]
    grid_points = df_grid['N1'] * df_grid['N2']  # N^2
    ax4.loglog(grid_points, df_grid['CPU_Time'], 'o-', label='CPU', linewidth=2, markersize=8, color='steelblue')
    if df_grid['GPU_Time'].notna().any():
        ax4.loglog(grid_points, df_grid['GPU_Time'], 's-', label='GPU', linewidth=2, markersize=8, color='coral')
    
    # 이론적 스케일링 참조선 (O(N^2))
    grid_ref = grid_points.values
    quadratic_ref = df_grid['CPU_Time'].iloc[0] * (grid_ref / grid_ref[0])
    ax4.loglog(grid_ref, quadratic_ref, '--', color='gray', alpha=0.5, label='O(N²) scaling')
    
    ax4.set_xlabel('Grid Points (N²) [log scale]', fontweight='bold')
    ax4.set_ylabel('Execution Time (seconds) [log scale]', fontweight='bold')
    ax4.set_title('Scaling Behavior (Log-Log Plot)')
    ax4.legend()
    ax4.grid(alpha=0.3, which='both')
    
    plt.tight_layout()
    plt.savefig('grid_scaling_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\n✅ 그래프가 grid_scaling_analysis.png로 저장되었습니다!")
else:
    print("⚠️ Grid 스케일링 데이터가 없습니다. Step 10을 먼저 실행하세요.")

## Step 12: 종합 성능 분석 리포트 🆕

In [ ]:
print("="*70)
print("     📊 ELS Pricer - 종합 성능 분석 리포트")
print("="*70)
print()

# 1. 기본 벤치마크 요약
csv_basic = '/content/els-pricer-cpp/build/cpu_vs_gpu_results.csv'
if os.path.exists(csv_basic):
    df_basic = pd.read_csv(csv_basic)
    print("### 1. 기본 벤치마크 (6개 그리드)")
    print(f"   - 평균 GPU 가속: {df_basic['Speedup'].mean():.2f}×")
    print(f"   - 최대 GPU 가속: {df_basic['Speedup'].max():.2f}×")
    print(f"   - GPU 우위 그리드: {(df_basic['Speedup'] > 1.0).sum()}/{len(df_basic)}")
    print()

# 2. Nt 스케일링 분석
csv_nt = '/content/els-pricer-cpp/build/nt_scaling_results.csv'
if os.path.exists(csv_nt):
    df_nt = pd.read_csv(csv_nt)
    print("### 2. Time Step (Nt) 스케일링 (100×100 고정)")
    print(f"   - 테스트 범위: Nt = {df_nt['Nt'].min()} ~ {df_nt['Nt'].max()}")
    if 'Speedup' in df_nt.columns:
        print(f"   - 평균 GPU 가속: {df_nt['Speedup'].mean():.2f}×")
        print(f"   - 최적 Nt: {df_nt.loc[df_nt['Speedup'].idxmax(), 'Nt']} (Speedup: {df_nt['Speedup'].max():.2f}×)")
    print(f"   - CPU 선형성: {(df_nt['CPU_Time'].iloc[-1] / df_nt['CPU_Time'].iloc[0]) / (df_nt['Nt'].iloc[-1] / df_nt['Nt'].iloc[0]):.3f} (1.0 = 완벽한 선형)")
    print()

# 3. Grid 스케일링 분석
csv_grid = '/content/els-pricer-cpp/build/grid_scaling_results.csv'
if os.path.exists(csv_grid):
    df_grid = pd.read_csv(csv_grid)
    print("### 3. Spatial Grid 스케일링 (Nt=1000 고정)")
    print(f"   - 테스트 범위: {df_grid['N1'].min()}×{df_grid['N1'].min()} ~ {df_grid['N1'].max()}×{df_grid['N1'].max()}")
    if 'Speedup' in df_grid.columns:
        print(f"   - 평균 GPU 가속: {df_grid['Speedup'].mean():.2f}×")
        print(f"   - 최적 Grid: {df_grid.loc[df_grid['Speedup'].idxmax(), 'Grid_Size']} (Speedup: {df_grid['Speedup'].max():.2f}×)")
    print(f"   - 최대 처리량: {(df_grid['Total_Points'] / df_grid['CPU_Time']).max() / 1e6:.1f}M points/sec (CPU)")
    if df_grid['GPU_Time'].notna().any():
        print(f"                  {(df_grid['Total_Points'] / df_grid['GPU_Time']).max() / 1e6:.1f}M points/sec (GPU)")
    print()

print("### 4. 핵심 인사이트")
print("   🎯 GPU는 대규모 그리드에서 특히 효과적")
print("   🎯 Time Step 증가는 CPU/GPU 모두 선형 스케일링")
print("   🎯 Spatial Grid 증가는 O(N²) 복잡도")
print("   🎯 메모리 제약으로 매우 큰 그리드는 GPU 성능 저하 가능")
print()

print("="*70)
print("     리포트 생성 완료!")
print("="*70)

## 📌 요약

### 테스트 완료!

이 확장 노트북에서:
1. ✅ GPU 환경 확인 (nvidia-smi, nvcc)
2. ✅ C++/CUDA 프로젝트 빌드 (CMake)
3. ✅ CPU vs GPU 기본 벤치마크 (6 grids)
4. 🆕 **Time Step (Nt) 스케일링 테스트** (200 → 2000)
5. 🆕 **Spatial Grid 스케일링 테스트** (100×100 → 700×700)
6. ✅ 성능 비교 및 시각화
7. ✅ 종합 분석 리포트

### 추가 테스트 내용
- **Nt 스케일링**: 100×100 고정, Nt를 200~2000까지 변경
- **Grid 스케일링**: Nt=1000 고정, 그리드를 100×100~700×700까지 변경

### 버그 수정 사항 (2024-11-14)
- 조기상환 로직 수정: 111.7원 → 103.9원
- 오차 개선: 7% → 0.5%

### 성능 (예상)
- **C++ CPU**: Python 대비 ~90-200× 빠름
- **C++ GPU**: CPU 대비 ~10-15× 추가 가속 (대규모 그리드)
- **총 가속**: Python 대비 ~900-3000×

### 결과 파일
- `cpu_vs_gpu_results.csv`: 기본 벤치마크 데이터
- `nt_scaling_results.csv`: Nt 스케일링 데이터
- `grid_scaling_results.csv`: Grid 스케일링 데이터
- `*.png`: 성능 비교 시각화 그래프

---

**프로젝트**: ELS Pricer C++/CUDA (Extended)  
**날짜**: 2024-11-17